# Snowflake Metadata Refresh Setup

## Import Python Libraries

In [1]:
import snowflake.connector
import os
import pandas as pd
import numpy as np
import pandas as pd
import pyarrow as pa
# pd.set_option('max_columns', 40)

## Set Snowflake Variables

In [2]:
# snowflake connection variables
snowflake_user = 'JMILLER'
snowflake_password = os.environ['BISNOWPASS']
snowflake_account = 'eh69371.east-us-2.azure'
snowflake_role = 'SYSADMIN'
snowflake_warehouse = 'COMPUTE_WH'
snowflake_database = 'UTIL_DB'
snowflake_schema = 'METADATA'

print('Using Notebook Variables:')
print('snowflake_user: ' + snowflake_user)
print('snowflake_password: ' + '***************')
print('snowflake_account: ' + snowflake_account)
print('snowflake_role: ' + snowflake_role)
print('snowflake_warehouse: ' + snowflake_warehouse)
print('snowflake_database: ' + snowflake_database)
print('snowflake_schema: ' + snowflake_schema)

Using Notebook Variables:
snowflake_user: JMILLER
snowflake_password: ***************
snowflake_account: eh69371.east-us-2.azure
snowflake_role: SYSADMIN
snowflake_warehouse: COMPUTE_WH
snowflake_database: UTIL_DB
snowflake_schema: METADATA


## Set Notebook Variables

In [3]:
tgt_table_schema = 'METADATA'
tgt_table_name = 'D_DATABASES'
               
print('Using Notebook Variables:')
print('tgt_table_schema: ' + tgt_table_schema)
print('tgt_table_name: ' + tgt_table_name)

Using Notebook Variables:
tgt_table_schema: METADATA
tgt_table_name: D_DATABASES


## Establish Snowflake Connection

In [4]:
ctx = snowflake.connector.connect(
    user = snowflake_user,
    password = snowflake_password,
    account = snowflake_account,
    role = snowflake_role,
    warehouse = snowflake_warehouse,
    database = snowflake_database,
    schema = snowflake_schema
    )
cur = ctx.cursor()

# Return Client
cur.execute("select CURRENT_CLIENT()")
one_row = cur.fetchone()
print('Snowflake Connection Successful')
print(one_row[0])

Snowflake Connection Successful
PythonConnector 2.5.1


## Get Table Mappings

In [5]:
sql = """
select SRC_TABLE_SCHEMA, SRC_TABLE_NAME, NATURAL_KEY, TYPE_2_COLUMNS, TYPE_0_COLUMNS, MAP_ADDED_COLUMNS_TAG
  from METADATA.MAP_TABLE
 where TGT_TABLE_SCHEMA = '""" + tgt_table_schema + """'
   and TGT_TABLE_NAME = '""" + tgt_table_name + """'
"""
print(sql)
cur.execute(sql)
one_row = cur.fetchone()
src_table_schema = one_row[0]
src_table_name = one_row[1]
natural_key = one_row[2]
type_2_columns = one_row[3]
type_0_columns = one_row[4]
map_added_columns_tag = one_row[5]

src_table_full = '"' + src_table_schema + '"."' + src_table_name + '"'
tgt_table_full = '"' + tgt_table_schema + '"."' + tgt_table_name + '"'

print('return values: ', src_table_schema, src_table_name, natural_key, type_2_columns, type_0_columns, map_added_columns_tag)
print('source table: ', src_table_full, '\nsource table: ', tgt_table_full)


select SRC_TABLE_SCHEMA, SRC_TABLE_NAME, NATURAL_KEY, TYPE_2_COLUMNS, TYPE_0_COLUMNS, MAP_ADDED_COLUMNS_TAG
  from METADATA.MAP_TABLE
 where TGT_TABLE_SCHEMA = 'METADATA'
   and TGT_TABLE_NAME = 'D_DATABASES'

return values:  INFORMATION_SCHEMA DATABASES DATABASE_NAME IS_TRANSIENT,RETENTION_TIME None info_schema_dim
source table:  "INFORMATION_SCHEMA"."DATABASES" 
source table:  "METADATA"."D_DATABASES"


## Get names for supplemental dimension columns

In [6]:
sql = """
select TGT_COLUMN_NAME
  from METADATA.MAP_ADDED_COLUMNS
 where MAP_ADDED_COLUMNS_TAG = '""" + map_added_columns_tag + """'
   and LOGICAL_COLUMN_NAME = 'row_is_current'
"""

print(sql)
cur.execute(sql)
one_row = cur.fetchone()
row_is_current = one_row[0]

sql = """
select TGT_COLUMN_NAME
  from METADATA.MAP_ADDED_COLUMNS
 where MAP_ADDED_COLUMNS_TAG = '""" + map_added_columns_tag + """'
   and LOGICAL_COLUMN_NAME = 'row_effective_date'
"""

print(sql)
cur.execute(sql)
one_row = cur.fetchone()
row_effective_date = one_row[0]

print('row_is_current: ', row_is_current, '\nrow_effective_date: ', row_effective_date)


select TGT_COLUMN_NAME
  from METADATA.MAP_ADDED_COLUMNS
 where MAP_ADDED_COLUMNS_TAG = 'info_schema_dim'
   and LOGICAL_COLUMN_NAME = 'row_is_current'


select TGT_COLUMN_NAME
  from METADATA.MAP_ADDED_COLUMNS
 where MAP_ADDED_COLUMNS_TAG = 'info_schema_dim'
   and LOGICAL_COLUMN_NAME = 'row_effective_date'

row_is_current:  ROW_IS_CURRENT 
row_effective_date:  ROW_EFFECTIVE_DATE


## Next

In [7]:
sql = ''
#df = cur.fetch_pandas_all()
#df

## Next

In [8]:
sql = ''
#df = cur.fetch_pandas_all()
#df